In [5]:
import xlsxwriter
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import xlwt

i,j,k, l = 0,0,0,0
row = 0
col = 0
N = 25 #number of files to read +1
Max = 1700 #maximum frequency of a peak to fit
Min = 1500 #minimum frequency of a peak to fit
conditions = [0.07, 79, 233240, 13.24, 1591.5, 68090.96, 15.55, 1566.9] #the initial conditions for the fit

xbook = xlsxwriter.Workbook('Output.xlsx')
def output(filename, sheet, list1, list2, x, y, z):
    book = xlwt.Workbook()
    sh = book.add_sheet(sheet)

#Lorentian or double-lorentian function 
def double_Lorentz(x, a, b, A, w, x_0, A1, w1, x_01):
    return a*x+b+(2*A/np.pi)*(w/(4*(x-x_0)**2 + w**2))+(2*A1/np.pi)*(w1/(4*(x-x_01)**2 + w1**2))

def Lorentz(x, a, b, A, w, x_0):
    return a*x+b+(2*A/np.pi)*(w/(4*(x-x_0)**2 + w**2))

temperatures = ['460K', '420K', '380K', '340K', '300K'] #names for new sheet
name = 'mapa_' #name of the txt file series of Raman spectroscopy maps
output = 'fit_output'

for m in range(0,N):
    data = np.loadtxt(name + str(m) +'.txt')
    print("reading file no" + str(m))
    x = data[:,0]
    y = data[:,1]
    wave = data[:,2]
    intensity = data[:,3]

    list_waves, list_intensities, list_x, list_y = [], [], [], []
    curr_series_beggining = 0
    wave_diff_bool = (wave[1:] - wave[:-1]) > 0

    beggining_indices = np.where(wave_diff_bool)
    beggining_indices = np.append(0, beggining_indices) #zero index
    beggining_indices = np.append(beggining_indices, len(wave)) #1st index

    for i in range(len(beggining_indices[:-1])): #extracting single spectrums from a mapping measurement file
        beginning = beggining_indices[i]
        ending = beggining_indices[i+1]
        if i==0: 
            current_wave = wave[beginning : ending+1]
            current_intensity = intensity[beginning : ending+1]
            current_x = x[beginning : ending+1]
            current_y = y[beginning : ending+1]
        else:
            current_wave = wave[beginning+1 : ending+1]
            current_intensity = intensity[beginning+1 : ending+1]
            current_x = x[beginning+1 : ending+1]
            current_y = y[beginning+1 : ending+1]
        
        list_waves.append(current_wave)
        list_intensities.append(current_intensity)
        list_x.append(current_x)
        list_y.append(current_y)
    
    n = 0
    index_peak_end=0 
    index_peak_begin=0
    for i in range(len(list_waves)):
        while list_waves[i][n] > Max: #maximum frequency of a peak to fit
            n = n+1
            index_peak_begin=n
        while list_waves[i][n] > Min: #minimum frequency of a peak to fit
            n = n+1
            index_peak_end=n
        n=1
     
    for i in range(len(list_waves)):
        intens_plot =list_intensities[i][index_peak_begin:index_peak_end]
        wave_plot =list_waves[i][index_peak_begin:index_peak_end]
        
    k+=1
    if m % 5 == 0:
        print('Writing measurement number: '+str(m)) #debug and control purpose only
        xsheet = xbook.add_worksheet('sheet_'+str(temperatures[l]))
        l+=1
        j = 0
        
    x_zeros, w_G, A_G = [], [], []
    
    for i in range(len(list_waves)):
            intens_plot =list_intensities[i][index_peak_begin:index_peak_end]
            wave_plot =list_waves[i][index_peak_begin:index_peak_end]
            parameters, covariance = curve_fit(double_Lorentz, wave_plot, intens_plot, conditions)
            a, b, A, w, x_0, A1, w1, x_01 = parameters
            x_fit = np.linspace(wave_plot.min(), wave_plot.max(), 1000)
            y_fit = double_Lorentz(x_fit, a, b, A, w, x_0, A1, w1, x_01)
            x_zeros.append(x_0) #x_0 - position of the fitted peak
            w_G.append(w) #full width at half maximum of the fitted peak
            A_G.append(A) #intensity of the fitted peak

    for idx, number in enumerate(x_zeros):
        xsheet.write(idx, j, x_zeros[idx])   
    j+=1

xbook.close()
print("Done!") 

reading file no0
Writing measurement number: 0
reading file no1
reading file no2
reading file no3
reading file no4
reading file no5
Writing measurement number: 5
reading file no6
reading file no7
reading file no8
reading file no9
reading file no10
Writing measurement number: 10
reading file no11
reading file no12
reading file no13
reading file no14
reading file no15
Writing measurement number: 15
reading file no16
reading file no17
reading file no18
reading file no19
reading file no20
Writing measurement number: 20
reading file no21
reading file no22
reading file no23
reading file no24
Done!
